## workflow (Hillebrandt)  
    
    https://github.com/HaukeHillebrandt/SPM_connectome
    
        Readme:
    
    SPM Steps

    Unpacking - run partial_unzipping_and_deletion_of_empty_folders.m - make sure you have the other functions in your path, this will unpack the tfMRI files needed for the SPM analysis

    Smoothing - smooth functional files with 4mm gaussian kernel

    1st level analysis - run batch_social_firstlevel_job_pmod.m and add_pmod_contrasts.m to create first level analysis based on parametric modulators (for DCM).

    You can also run batch_social_firstlevel_job_contrasts.m if you need t-contrasts instead of parametric modulators.

    2nd Level analysis - here you can run an ANOVA which allows you to do 2nd level conjunction analysis but is less powerful or bring 1-level t-contrasts to the 2nd level (more statistical power)
    DCM Steps

    Extract VOIs for the DCM with VOI_extraction_pmod_model_2.m. You may want to check whether the voxels extract do not overlap with voi_overlap_check.m

    Build and estimate the full model DCM with build_DCM_public.m

    run_post_hoc_model.m to run exhaustive DCM
    
    

----------

##  workflow before DCM
    
    0. Предобработанные данные (D)
        Out:
        1) D - nifti данные 
        2) n - длина временных рядов
        3) p - кол-во участников эксперимента
        4) k - общее число вокселей
    
    1.Preprocessing (spatial smoothing)
        In:
        1) D
        2) gaussian kernel 4*4*4mm
        Out:
        1) D*
        (Y-набор временных рядов для каждого вокселя каждого участника) (p*(n*k))
        
        
    2.Creating regressors and contrasts for GLM
        In:
        1) 2 экспериментальных регрессора (all-motion/no-motion, animate/inanimate)
        2) HRF
        3) constant session effects
        4) regressors representing residual movement-related artifacts and their derivatives
        Out:
        1) m - общее число регрессоров
        2) X - дизайн-матрица (n*m)
        3) c - контрасты 
        
        
    3. VOIs selection + summarizing timeseries from VOIs
        In:
        1) X (n*m)
        2) Y (p*(n*k))
        3) assumptions about ε, η
        4) с
        5) p
        6) the radii of the VOI spheres
        7) the search radii for local maxima from the group analysis
        8) anatomy toolbox (?)
        9) s - число VOI
        Out:
        1) timeseries for s VOIs

        *(test: все ли участники имеют значимую активность в этих voi)
        
        
            1. GLM
                Fit a model for each subject data
                Define the effect of interest for each subject with a contrast
                Construct contrast images for each subject
                Conduct a t-test on the contrast images

            1.1 First-level analysis (within subjects analysis)
                In: 
                1), 2), 3.1), 4), 5)

                Yp = Xp*𝛃p + εp     ⇛ Y = X*𝛃 + ε 
                ε ~ N(0,V)                
                
                Out1:
                1) 𝛃 (fitting)

                c - t-контасты (assigning weights to each beta 𝛃i  to test hypothesis):
                H0:cT*𝛃 = 0
                (define whether task related activity is present in voxel)
                t-tests across all voxels
                ⇛ localization of brain activity

                Out2:
                2) contrast images for each subjects (t-map)
                3) Results at each voxel:
                • Contrast Value
                • Contrast Value Variance
                • p-value (Volume of Activation)
                (active voxels for each subject)

            1.2 Second-level analysis (group analysis)
                In: 
                1), 2), 3.2), 4), 5), 9)
                10) 𝛃

                𝛃 = Xg*𝛃g + η   
                η ~ N(0,Vg)

                Out1: 
                1) 𝛃g
                T-tests on contrasts

                Out2: 
                2) active voxels in group 

            2. VOI selection
                In:
                6), 7), 8), 9)
                11)  active voxels in group

                “The radii of the VOI spheres were 6 mm and the search radii for local maxima from the group analysis were restricted to 20 mm. [All voxels contributing to the eigenvariates were significant at p< 0.05 uncorrected and adjusted at p < 0.05 for the effects of interest (i.e. only for those regressors that were used in the DCMs for input or modulation)].
                The first volume of interest (VOI) was based on maxima in the most active cluster of the All motion contrast (which was Animate and Inanimate motion over the implicit fixation baseline). The second VOI was extracted at the local maxima of the most active clusters in each hemisphere based on the results of a conjunction analysis. The conjunction used here was the effect of the contrast [All Motion > Fixation Cross] & (logical AND) the contrast [Animate > Inanimate Motion] – i.e. [All motion > Fixation cross & Animate – inanimate motion]. The conjunction was performed to consider areas more active in Animate vs. Inanimate motion, but only in motion sensitive areas (activated by any type of motion). We used the more conservative test, testing against the conjunction null, instead of testing against the global null.”

                Out: 
                1) s VOI (s наборов вокселей)

            3. Assigning maxima to region names by the SPM anatomy toolbox.

            4. Summarizing (extracting) timeseries from VOIs for DCM

                In:
                1) s VOIs 

                “Timeseries from VOIs associated with the contrasts were summarised using the SPM12b Eigenvariate toolbox: we extracted each participant's principal eigenvariate around the participant-specific local maxima activations nearest to the peak voxel of the group (between subject) GLM analysis” 

                Out:
                1) ts for s VOIs

